# Sentiment Analysis on Glassman Issue
Greg Glassman made an insensitive comment relating to George Floyd during the BLM movement in June 2020. This notebook is a sentiment analysis on documents mentioning glassman, castro, or hq

In [13]:
import praw
import pandas as pd
import numpy as np
import pickle

import reddit as red

In [25]:
#import data
picklefile_name = './reddit_pkl_data/df_praw_full_blm.pkl'
with open(picklefile_name, 'rb') as picklefile: 
    df_praw = pickle.load(picklefile)

In [31]:
df_praw.shape

(65790, 14)

In [28]:
#df_praw.drop(columns='content_categories', inplace=True)

In [32]:
mask = (df_praw.glassman > 0) | (df_praw.castro > 0) | (df_praw.hq > 0)
df_blm = df_praw[mask]
df_blm.shape

(1464, 14)

In [85]:
df_blm.loc[268].full_text

'do you agree with crossfits endorsement of zone over paleo i am a trainer and have been to about   dozen boxes all over the world not just the us and have seen a much bigger backing from the affiliates of paleo  i am a trainer at a box and i push paleo over zone for a few reasons i did zone for a year and while i saw results it was almost crippling socially and mentally counting measuring eating at particular times ect  i never really felt better or saw improvements in the gym like i have with paleo paleo is more than just losing weight its been performance  as a trainer i dont like suggesting zone especially to young women who may or may not have had issues with eating disorders i feel like paleo is much more relaxed than zone whereas zone brings back some neurosis about food  do you think hq is out of touch by backing zone so much most of the top athletes who do zone do a paleo zone with   fats and low carb anyways  '

In [112]:
df_orig.loc[1015].full_text

'Do you agree with CrossFits endorsement of Zone over Paleo? I am a trainer and have been to about 2 dozen boxes all over the world (not just the US) and have seen a much bigger backing from the affiliates of paleo.  I am a trainer at a box and I push Paleo over Zone for a few reasons. I did Zone for a year and while i saw results it was almost crippling socially and mentally. Counting, measuring, eating at particular times ect...  I never really felt better or saw improvements in the gym like i have with paleo. Paleo is more than just losing weight its been performance.  As a trainer i dont like suggesting zone, especially to young women who may or may not have had issues with eating disorders. I feel like paleo is much more relaxed than zone, whereas zone brings back some neurosis about food.  Do you think HQ is out of touch by backing zone so much? Most of the top athletes who do zone do a paleo zone with 5x fats and low carb anyways.  '

In [35]:
#import original df with original wording
picklefile_name = './reddit_pkl_data/2008_2021feb19_df.pkl'
with open(picklefile_name, 'rb') as picklefile: 
    df_orig = pickle.load(picklefile)

In [106]:
df_orig = red.clean_praw_input(df_orig)
df_orig = red.clean_submissions(df_orig)
df_orig.drop(columns=['created', 'content_categories', 'is_meta'], inplace=True)
df_orig.shape

(66948, 9)

In [37]:
import re

#remove http/https links
links = lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE)
#embedded links
links2 = lambda x: re.sub(r'`&.*link rel=.*”.*;', '', x, flags=re.MULTILINE)
#&amp*
amps = lambda x: re.sub(r'&amp;.*;', '', x, flags=re.MULTILINE)
#remove links in () or []
links3 = lambda x: re.sub(r'[\(\[]https?:\/\/.*[\)\]]', '', x, flags=re.MULTILINE)

#remove slashes
slashes = lambda x: re.sub(r'\/', ' ', x, flags=re.MULTILINE)

df_orig['full_text'] = df_orig.full_text.map(links).map(links2).map(amps).map(links3).map(slashes)

AttributeError: 'DataFrame' object has no attribute 'full_text'

In [111]:
# alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
# punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())
new_line = lambda x: x.replace('\n', ' ')
# emojis = lambda x: x.encode('ascii', 'ignore').decode('ascii')

df_orig['full_text'] = df_orig.full_text.map(new_line)

In [39]:
#import df_orig and df_blem
picklefile_name = './reddit_pkl_data/sentiment_df_blm.pkl'
with open(picklefile_name, 'rb') as picklefile: 
    df_blm = pickle.load(picklefile)

In [40]:
blm_idx_list = list(df_blm['index'])

In [41]:
df_filt = df_orig.loc[blm_idx_list]

In [42]:
df_filt.shape

(1464, 14)

In [43]:
df_filt

,author,created_utc,id,is_video,num_comments,score,selftext,thumbnail,title,url,created,is_meta,content_categories,upvote_ratio
1015,Skiingjoo,1305388555,hbabg,NaN,10,3,I am a trainer and have been to about 2 dozen ...,self,Do you agree with CrossFits endorsement of Zon...,http://www.reddit.com/r/crossfit/comments/hbab...,1.305417e+09,NaN,NaN,NaN
777,kurian,1312482869,j921n,NaN,41,6,"So I've been training in two boxes, as it's a ...",self,Crossfit heading in a bad direction.,http://www.reddit.com/r/crossfit/comments/j921...,1.312512e+09,NaN,NaN,NaN
667,chiuondis,1315102696,k40pq,NaN,10,4,"I know he started crossfit and all, but he's r...",self,Does anyone know what's up with Glassman?,http://www.reddit.com/r/crossfit/comments/k40p...,1.315131e+09,NaN,NaN,NaN
523,badmonkey283,1318180020,l64vn,NaN,11,10,"Hi, I have a question for all you CrossFitting...",self,question: Would you use...,http://www.reddit.com/r/crossfit/comments/l64v...,1.318209e+09,NaN,NaN,NaN
504,Breeegz,1318616085,lcbfb,NaN,6,14,,http://thumbs.reddit.com/t3_lcbfb.png,Drywall Crossfit Trolls HQ,http://www.forgingelitesarcasm.com/2011/10/int...,1.318645e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63398,Do_Not_Upvote_69,1609951039,krrevv,False,21,2,,default,Dave Castro stirring the pot,https://www.instagram.com/p/CJtbIupAmld/?hl=en,1.609980e+09,False,NaN,1.0
62962,Marvin_rock,1611964721,l88g2b,False,41,1,,https://b.thumbs.redditmedia.com/fVrBfh8RR1WVZ...,Castro just confirmed only a single 50/35# dum...,https://www.youtube.com/watch?list=TLGG9llUNal...,1.611994e+09,False,NaN,1.0
62930,IntroductionNo1067,1612064549,l93nn7,False,1,1,,https://b.thumbs.redditmedia.com/H4FFLwBGT-Jyj...,"Meeting Between Masters Fitness Collective, Ca...",https://morningchalkup.com/2020/12/13/meeting-...,1.612093e+09,False,NaN,1.0
62886,angrytongan,1612226493,laj256,False,10,30,[wodscrape.com.au](https://wodscrape.com.au) i...,self,[wodscrape] Shutting it down,https://www.reddit.com/r/crossfit/comments/laj...,1.612255e+09,False,NaN,1.0


In [22]:
# pip install vaderSentiment

In [113]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [120]:
text_list = df_filt.full_text
text_list

1015     Do you agree with CrossFits endorsement of Zon...
777      Crossfit heading in a bad direction. So I've b...
667      Does anyone know what's up with Glassman? I kn...
523      question: Would you use...  Hi, I have a quest...
504                            Drywall Crossfit Trolls HQ 
                               ...                        
63398                        Dave Castro stirring the pot 
62962    Castro just confirmed only a single 50 35# dum...
62930    Meeting Between Masters Fitness Collective, Ca...
62886    [wodscrape] Shutting it down [wodscrape.com.au...
62740    Wait, Can They Save CrossFit? Updated thoughts...
Name: full_text, Length: 1464, dtype: object

In [121]:
def sentiment_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [122]:
scores_list = []
for text in text_list:
    scores_list.append(sentiment_scores(text))

In [123]:
df_scores = pd.DataFrame(scores_list)

In [125]:
mask = (df_scores.pos > 0.5)
df_scores[mask]

,neg,neu,pos,compound
18,0.0,0.435,0.565,0.3818
136,0.0,0.448,0.552,0.5719
215,0.0,0.462,0.538,0.8885
265,0.0,0.370,0.630,0.5267
284,0.0,0.408,0.592,0.4404
872,0.0,0.341,0.659,0.8286
1005,0.0,0.494,0.506,0.6249
1275,0.0,0.455,0.545,0.0516


In [126]:
df_scores.shape

(1464, 4)

In [128]:
df_filt.shape

(1464, 9)

In [131]:
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [206]:
#pip install ibm_watson

In [277]:
watson_tone_service_api_key = 'QC4wbP3LdtXMDF3784jEMnTi_vR3IzahfPE7qgbRHc81'
watson_tone_service_url = 'https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/db975326-38f1-4341-a80b-c29a17807aa2'

authenticator = IAMAuthenticator(watson_tone_service_api_key)
tone_analyzer = ToneAnalyzerV3(authenticator=authenticator, 
                               version='2017-09-21')
                               
tone_analyzer.set_service_url(watson_tone_service_url)

tone_analyzer.set_disable_ssl_verification(True)

In [279]:
text = df_filt.full_text.iloc[0]
text

'Do you agree with CrossFits endorsement of Zone over Paleo? I am a trainer and have been to about 2 dozen boxes all over the world (not just the US) and have seen a much bigger backing from the affiliates of paleo.  I am a trainer at a box and I push Paleo over Zone for a few reasons. I did Zone for a year and while i saw results it was almost crippling socially and mentally. Counting, measuring, eating at particular times ect...  I never really felt better or saw improvements in the gym like i have with paleo. Paleo is more than just losing weight its been performance.  As a trainer i dont like suggesting zone, especially to young women who may or may not have had issues with eating disorders. I feel like paleo is much more relaxed than zone, whereas zone brings back some neurosis about food.  Do you think HQ is out of touch by backing zone so much? Most of the top athletes who do zone do a paleo zone with 5x fats and low carb anyways.  '

In [280]:
# text = 'Team, I know that times are tough! Product '\
#     'sales have been disappointing for the past three '\
#     'quarters. We have a competitive product, but we '\
#     'need to do a better job of selling it!'

tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
print(json.dumps(tone_analysis, indent=2))

{
  "document_tone": {
    "tones": [
      {
        "score": 0.552201,
        "tone_id": "joy",
        "tone_name": "Joy"
      },
      {
        "score": 0.916397,
        "tone_id": "tentative",
        "tone_name": "Tentative"
      }
    ]
  },
  "sentences_tone": [
    {
      "sentence_id": 0,
      "text": "Do you agree with CrossFits endorsement of Zone over Paleo?",
      "tones": []
    },
    {
      "sentence_id": 1,
      "text": "I am a trainer and have been to about 2 dozen boxes all over the world (not just the US) and have seen a much bigger backing from the affiliates of paleo.",
      "tones": []
    },
    {
      "sentence_id": 2,
      "text": "I am a trainer at a box and I push Paleo over Zone for a few reasons.",
      "tones": [
        {
          "score": 0.538448,
          "tone_id": "analytical",
          "tone_name": "Analytical"
        }
      ]
    },
    {
      "sentence_id": 3,
      "text": "I did Zone for a year and while i saw results it wa

In [281]:
tones = tone_analyzer.tone(text, content_type='text/plain;charset=utf-8').get_result()
# tones_df = pd.io.json.json_normalize(tones['document_tone'], 'tones')

In [284]:
tone_analyzer.tone(text, content_type='text/plain;charset=utf-8').get_result()

{'document_tone': {'tones': [{'score': 0.552201,
    'tone_id': 'joy',
    'tone_name': 'Joy'},
   {'score': 0.916397, 'tone_id': 'tentative', 'tone_name': 'Tentative'}]},
 'sentences_tone': [{'sentence_id': 0,
   'text': 'Do you agree with CrossFits endorsement of Zone over Paleo?',
   'tones': []},
  {'sentence_id': 1,
   'text': 'I am a trainer and have been to about 2 dozen boxes all over the world (not just the US) and have seen a much bigger backing from the affiliates of paleo.',
   'tones': []},
  {'sentence_id': 2,
   'text': 'I am a trainer at a box and I push Paleo over Zone for a few reasons.',
   'tones': [{'score': 0.538448,
     'tone_id': 'analytical',
     'tone_name': 'Analytical'}]},
  {'sentence_id': 3,
   'text': 'I did Zone for a year and while i saw results it was almost crippling socially and mentally.',
   'tones': [{'score': 0.5538,
     'tone_id': 'tentative',
     'tone_name': 'Tentative'}]},
  {'sentence_id': 4,
   'text': 'Counting, measuring, eating at pa

In [227]:
test = pd.DataFrame(tones['document_tone']['tones'])

In [239]:
test[['add', 'this']] = pd.DataFrame([['row', 'info']], index=test.index)

In [240]:
test

,score,tone_id,tone_name,add,this
0,0.664063,sadness,Sadness,row,info
1,0.883404,analytical,Analytical,row,info


In [205]:
list(tones['document_tone']['tones'][0].values())

[0.552201, 'joy', 'Joy']

In [235]:
data = [{'state': 'Florida',
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]
pd.json_normalize(data, 'counties', ['state', 'shortname',
                                           ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


In [234]:
for idx, row in df_part.iterrows():
    print(type(row))
    print(type([row]))

<class 'pandas.core.series.Series'>
<class 'list'>
<class 'pandas.core.series.Series'>
<class 'list'>
<class 'pandas.core.series.Series'>
<class 'list'>
<class 'pandas.core.series.Series'>
<class 'list'>
<class 'pandas.core.series.Series'>
<class 'list'>


In [207]:
# df_part = df_filt.iloc[:5]
df_filt['vote_score'] = df_filt['score']
df_filt.drop(columns='score', inplace=True)

In [208]:
post_tones = pd.DataFrame()

for idx, row in df_filt.iterrows():
    tones = tone_analyzer.tone(row['full_text'], content_type='text/plain;charset=utf-8').get_result()
    tones_df = pd.io.json.json_normalize(tones['document_tone'], 'tones')
    
    tones_df[df_filt.columns.tolist()] = pd.DataFrame([row], index=tones_df.index)
    post_tones = post_tones.append(tones_df)

In [285]:
post_tones

,score,tone_id,tone_name,author,id,num_comments,url,upvote_ratio,time,full_text,media,vote_score
0,0.552201,joy,Joy,Skiingjoo,hbabg,10,http://www.reddit.com/r/crossfit/comments/hbab...,NaN,2011-05-14 16:55:55,Do you agree with CrossFits endorsement of Zon...,none,3
1,0.916397,tentative,Tentative,Skiingjoo,hbabg,10,http://www.reddit.com/r/crossfit/comments/hbab...,NaN,2011-05-14 16:55:55,Do you agree with CrossFits endorsement of Zon...,none,3
0,0.570424,sadness,Sadness,kurian,j921n,41,http://www.reddit.com/r/crossfit/comments/j921...,NaN,2011-08-04 19:34:29,Crossfit heading in a bad direction. So I've b...,none,6
1,0.509031,analytical,Analytical,kurian,j921n,41,http://www.reddit.com/r/crossfit/comments/j921...,NaN,2011-08-04 19:34:29,Crossfit heading in a bad direction. So I've b...,none,6
0,0.844556,analytical,Analytical,chiuondis,k40pq,10,http://www.reddit.com/r/crossfit/comments/k40p...,NaN,2011-09-04 03:18:16,Does anyone know what's up with Glassman? I kn...,none,4
...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.617335,joy,Joy,angrytongan,laj256,10,https://www.reddit.com/r/crossfit/comments/laj...,1.0,2021-02-02 00:41:33,[wodscrape] Shutting it down [wodscrape.com.au...,none,30
1,0.671669,fear,Fear,angrytongan,laj256,10,https://www.reddit.com/r/crossfit/comments/laj...,1.0,2021-02-02 00:41:33,[wodscrape] Shutting it down [wodscrape.com.au...,none,30
2,0.904259,tentative,Tentative,angrytongan,laj256,10,https://www.reddit.com/r/crossfit/comments/laj...,1.0,2021-02-02 00:41:33,[wodscrape] Shutting it down [wodscrape.com.au...,none,30
0,0.664063,sadness,Sadness,Thegooseontheisland,liez7q,26,https://www.rocketcommunityfitness.com/post/wa...,1.0,2021-02-12 16:47:52,"Wait, Can They Save CrossFit? Updated thoughts...",NaN,1


In [286]:
picklefile_name = 'post_tones_df.pkl'
with open(picklefile_name, 'wb') as picklefile:
    pickle.dump(post_tones, picklefile)

In [244]:
#export data to CSV for viz in tableau
post_tones.to_csv("cf_submission_tones.csv")

In [ ]:
tone_analyzer.set_detailed_response(True)
response = tone_analyzer.methodName(parameters)
# Access response from methodName
print(json.dumps(response.get_result(), indent=2))
# Access information in response headers
print(response.get_headers())
# Access HTTP response status
print(response.get_status_code())

In [3]:
#import data
picklefile_name = 'post_tones_df' + '.pkl'
with open(picklefile_name, 'rb') as picklefile: 
    post_tones_df = pickle.load(picklefile)

In [12]:
post_tones_df[post_tones_df.tone_id == 'joy'].iloc[5].full_text

"To anyone starting a home crossfit gym. I see and hear of people wanting to build their own garage gym all the time.  Almost every time people want to buy kettlebells 1st for their crossfit home gym.  I don't know why everyone wants to buy kettlebells first, or why most people think this is the most crucial equipment for crossfit.   buy barbells first.  No questions asked, this is the most important part to any home gym, but especially crossfit.   yes, kettlebells are great, but they are a much smaller piece to the puzzle. There is an awesome article on the hq website that explains what you should buy and in what order, but i just wanted to throw it out there to stop buying kettlebells 1st.   Eventually, yes buy them, but if you want the most bang for your buck right now,  buy yourself a barbell and some weights.    *Edit - damn autocorrect"